In [19]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import roc_auc_score
from clockwork_helperfunc import *
from clockwork_helperfunc import evaluation
import clockwork_helperfunc 
from imp import reload  
reload(clockwork_helperfunc)

#configuration
batch_size = 5
num_epochs = 50
number_units = 7

### Data

In [20]:
training = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_train_data.p')
val = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_valid_data.p')

In [21]:
patient_feature_train, label_train, label_time_train = downsample(training, proportional = 2)

In [22]:
patient_feature_val = val[0] #before cleaning, original data
label_val = val[1]
label_time_val = val[2]

In [5]:
sum(label_time_train), sum(label_train*label_time_train)

(521392, 237789)

In [6]:
Counter(label_train), Counter(label_val)

(Counter({0: 3535, 1: 1765}), Counter({0: 4051, 1: 549}))

In [7]:
len(patient_feature_train)

5300

### small_try

In [5]:
idx = np.random.randint(0, len(patient_feature_train),10)
patient_feature_train_sub, label_train_sub, label_time_train_sub = patient_feature_train.copy(), label_train.copy(), label_time_train.copy()
patient_feature_train, label_train, label_time_train = patient_feature_train_sub[idx], label_train_sub[idx], label_time_train_sub[idx]

In [6]:
#patient_feature_train = [patient_feature_train[i][:i+1, :5] for i in range(len(patient_feature_train))]

### Model

In [23]:
### Model
#forward
#cell_class, step
class Clock_NN(nn.Module):
    def __init__(self, scale,batch_size, group_size = 1, activation_fun =nn.Tanh, mean = 0, std = 1, input_dim = 48,mode = 'shift'):
        super(Clock_NN, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        activation_function
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.scale = scale
        self.group_size = group_size
        self.batch_size = batch_size
        self.mode = mode
        if mode == 'original':
            self.num_units = len(self.scale)*self.group_size
            self.index_li = {self.scale[i]: i for i in range(len(self.scale))}
        elif mode == 'shift':
            self.num_units = sum(self.scale)*self.group_size
            self.index_li = {i:i-1 for i in self.scale}
            
        self.class_dim = 2
        self.input_dim = input_dim
        self.linear_h = nn.Linear(self.num_units,self.num_units)
        self.linear_o = nn.Linear(self.num_units,self.class_dim)
        self.linear_i = nn.Linear(self.input_dim, self.num_units)
        self.activation_fun = activation_fun
        self.connect = torch.from_numpy(block_tri(self.group_size, self.scale, self.num_units, self.mode)).float()
        self.time_step = 0
        
        self.initial_weights(mean, std)
        
        #the connectivity, when we disabled the weight, this should not change
        self.linear_h.weight.data = self.linear_h.weight.data*self.connect#here needs transpose since previously left multiplication, activate mtrx doesn't need as rewrite and select cols.
        #self.linear_i.weight.data = self.linear_i.weight.data
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        '''     
        #sequence = sequence.view(48,-1)when this is only one batch
        hidden_output = []
        length = sequence.size()[1]
        logit = []
        for i in range(length):
            #print('this is the timestep ' + str(self.time_step))
            self.time_step += 1
            #backwards, want discharge/dead time aligns
            #print(sequence[:,:,-i].size())#would be batch*48
            hidden = self.CW_RNN_Cell(sequence[:,i,:].contiguous(), hidden)
            hidden_output.append(hidden)#become batch_size x hidden_dim
            out = self.linear_o(hidden)
            logit.append(F.log_softmax(out))
        return hidden_output, logit 
    #a list of predictions of each step(till the largest, each one is batch * hidden_dim); 
    #hidden_output is a list of output prediction, each one is batch * two
            
                
    def CW_RNN_Cell(self, x_input, hidden):
        '''
        x_input: number_feature x batch vector, representing one time stamp
        hidden: output of the last cell, should be hidden_dim(i.e. num_units) x batch
        '''
        #which time bloack to change
        activate = activate_index(self.time_step, self.num_units, self.group_size, self.scale,self.index_li,batch_size,self.mode, self.input_dim)
        activate_re = torch.from_numpy(np.ones((self.batch_size,self.num_units))).float() - activate

        hidden_next = self.linear_h(hidden) + self.linear_i(x_input) #should be batch_size x hidden_dim       
        hidden_next.data = activate*hidden_next.data + activate_re*hidden.data
        hidden_next = self.activation_fun(hidden_next)
        return hidden_next

    def init_hidden(self):
        h0 = Variable(torch.zeros(self.batch_size,self.num_units))
        return h0
        
    def initial_weights(self, mean, std):
        lin_layers = [self.linear_h, self.linear_o, self.linear_i]
        for layer in lin_layers:
            layer.weight.data.normal_(mean, std**2)
            layer.bias.data.fill_(0) 

In [ ]:
### Training original
model = Clock_NN([1,2,4,8,16], batch_size, group_size = 5, activation_fun = F.tanh, mean = 0, std = 0.1, input_dim = 40, mode = 'original')

loss = torch.nn.NLLLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(model.parameters(), lr=0.000005)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size,patient_feature_train, label_train, label_time_train,patient_feature_val, label_val, label_time_val)
for epoch in range(20):
    for step, (data, label, label_time) in enumerate(train_loader):
        data, label = Variable(data), Variable(label)
        model.zero_grad()
        hidden= model.init_hidden()
        model.time_step = 0
        hidden, output = model(data, hidden)
        #print(hidden[0].size())
        #print(output[0].size())
        #now get a list of hidden and a list of outputs
        
        #patient level
        label = label.transpose(0,1).contiguous().view(-1) 
        #should be flatten, batch_size x hidden. transpose due to below order, was batch, seq => follow up 2 down. get size batch*seq          
        output = torch.stack(output, dim=1).view(-1, 2) 
        #print(output[-1])
            
        lossy = loss(output, label)
        lossy.backward()
        model.linear_h.weight.grad.data = model.linear_h.weight.grad.data*model.connect
        optimizer.step()
        
        if step%4 ==0 :
            print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
            acc0, acc1, val_acc, auc = evaluation(validation_loader, model)
            print('accuracy_on_validation: {}, the acc for LIVE is {}, the acc for DEAD is {}'.format(val_acc, acc0, acc1)) 
            print('the auc is ' + str(auc))
    #accuracy_list.append(val_acc)


Epoch: 0; Loss: 0.6930732131004333
accuracy_on_validation: 0.6002173913043478, the acc for LIVE is 0.6184886280264124, the acc for DEAD is 0.45401174168297453
the auc is 0.536250184855
Epoch: 0; Loss: 0.692920446395874
accuracy_on_validation: 0.6021739130434782, the acc for LIVE is 0.6187331865981903, the acc for DEAD is 0.46966731898238745
the auc is 0.54420025279
Epoch: 0; Loss: 0.6928211450576782
accuracy_on_validation: 0.6086956521739131, the acc for LIVE is 0.6292492051846417, the acc for DEAD is 0.44422700587084146
the auc is 0.536738105528
Epoch: 0; Loss: 0.6931390762329102
accuracy_on_validation: 0.6121739130434782, the acc for LIVE is 0.6336512594766447, the acc for DEAD is 0.44031311154598823
the auc is 0.536982185511
Epoch: 0; Loss: 0.6928198337554932
accuracy_on_validation: 0.6, the acc for LIVE is 0.6184886280264124, the acc for DEAD is 0.4520547945205479
the auc is 0.535271711273
Epoch: 0; Loss: 0.6931297779083252
accuracy_on_validation: 0.6060869565217392, the acc for LI

In [14]:
def evaluation(data_loader, model, mode = 'novel'):
    '''
    This is the evaluation of patient level prediction.
    '''
    label_list = []
    output_list = []
    model.eval()
    
    for i, (data, label, time_list) in enumerate(data_loader):
        data = Variable(data)
        hidden= model.init_hidden()
        #print(data.size()) #[5, 75, 48]
        #print(time_list) #which is a list
        if mode == 'novel':
            hidden, ori_output = model(data, hidden)
            ##print('the hidden size is ')
            #print(hidden[0].data.size()) #batch_size * hidden dimension
            #output is a list of prediction, everyone is batch * two
            #print('output length' + str(len(output)))
            #print('output single element size' + str(output[0].size()))
            output = [np.argmax(ori_output[time_list[i] - 1][i].data.numpy()) for i in range(data.size()[0])]
            #batch size, each one pick the element
            #each prediction for each batch member
            #print(output[0]) 
        else:
            output = model(data, hidden).data.numpy() #should be [torch.FloatTensor of size batch * 1 * 3]
        #now get a list of hidden and a list of outputs
        #print(output)
        label = label[:,0].numpy()
        #print(label == output)
        #label = label.transpose(0,1).contiguous().view(-1).data.numpy()

        label_list.extend(label)
        output_list.extend(output)
    
    label_list = np.array(label_list)
    output_list = np.array(output_list)
    
    one_idx = np.where(label_list == 1)[0]
    zero_idx = np.where(label_list == 0)[0]
    
    label_ones = label_list[one_idx] #both are array, can select index like this
    label_zeros = label_list[zero_idx]
    pre_ones = output_list[one_idx]#now softmax, turn into class
    pre_zeros = output_list[zero_idx]
        

    acc0 = sum(pre_zeros == label_zeros)/len(pre_zeros)
    acc1 = sum(pre_ones ==label_ones)/len(pre_ones)
    acc =  (sum(pre_ones == label_ones) + sum(pre_zeros == label_zeros))/(len(pre_ones) + len(pre_zeros))
    #print(type(label_ones), type(label_zeros), type(pre_ones),type(pre_zeros))
    auc = roc_auc_score(list(label_ones) + list(label_zeros), list(pre_ones) + list(pre_zeros))
    model.train()
    return acc0, acc1,acc,auc



In [15]:
np.array([1,2,3,4])[np.array([1,2,3])]

array([2, 3, 4])

In [16]:
evaluation(validation_loader, model, mode = 'novel')

KeyboardInterrupt: 

### debug

In [ ]:
### Model
#forward
#cell_class, step
class Clock_NN(nn.Module):
    def __init__(self, scale,batch_size, group_size = 1, activation_fun =nn.Tanh, mean = 0, std = 1, input_dim = 48,mode = 'shift'):
        super(Clock_NN, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        activation_function
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.scale = scale
        self.group_size = group_size
        self.batch_size = batch_size
        self.mode = mode
        if mode == 'original':
            self.num_units = len(self.scale)*self.group_size
            self.index_li = {self.scale[i]: i for i in range(len(self.scale))}
        elif mode == 'shift':
            self.num_units = sum(self.scale)*self.group_size
            self.index_li = {i:i-1 for i in self.scale}
            
        self.class_dim = 2
        self.input_dim = input_dim
        self.linear_h = nn.Linear(self.num_units,self.num_units)
        self.linear_o = nn.Linear(self.num_units,self.class_dim)
        self.linear_i = nn.Linear(self.input_dim, self.num_units)
        self.activation_fun = activation_fun
        self.connect = torch.from_numpy(block_tri(self.group_size, self.scale, self.num_units, self.mode)).float()
        self.time_step = 0
        print('----------------print connectivity-------------------------------------------------------')
        print(self.connect)
        
        
        self.initial_weights(mean, std)
        
        #the connectivity, when we disabled the weight, this should not change
        self.linear_h.weight.data = self.linear_h.weight.data*self.connect#here needs transpose since previously left multiplication, activate mtrx doesn't need as rewrite and select cols.
        #self.linear_i.weight.data = self.linear_i.weight.data
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        '''     
        #sequence = sequence.view(48,-1)when this is only one batch
        hidden_output = []
        length = sequence.size()[1]
        logit = []
        for i in range(1,length+1):
            print('this is the timestep ' + str(self.time_step))
            self.time_step += 1
            #backwards, want discharge/dead time aligns
            #print(sequence[:,:,-i].size())#would be batch*48
            hidden = self.CW_RNN_Cell(sequence[:,-i,:].contiguous(), hidden)
            hidden_output.append(hidden)#become batch_size x hidden_dim
            out = self.linear_o(hidden)
            print('---------------- linear layer weight-------------------------------')
            print(self.linear_o.weight.data)
            print('---------------- out for prediction linear_o(hidden from next step)-------------------------------')
            print(out[:,0] == out[:,1])#0 means false. test whether it cares about how many zeros are after this
            #print(F.softmax(out))
            print(hidden)
            print(out)
            print(F.log_softmax(out))
            #print(out.size())#should be batch x 2 (for one timestep)
            logit.append(F.log_softmax(out))
        return hidden_output, logit
            
                
    def CW_RNN_Cell(self, x_input, hidden):
        '''
        x_input: number_feature x batch vector, representing one time stamp
        hidden: output of the last cell, should be hidden_dim(i.e. num_units) x batch
        '''
        print('---------------- hidden step -------------------------------------------------------')
        print(hidden)

        print('---------------- x_input -------------------------------------------------------')
        print(x_input)
        
        #which time bloack to change
        activate = activate_index(self.time_step, self.num_units, self.group_size, self.scale,self.index_li,batch_size,self.mode, self.input_dim)
        activate_re = torch.from_numpy(np.ones((self.batch_size,self.num_units))).float() - activate
        
        #print('activate ' + str(activate.size()))
        #print('activate_re ' + str(activate_re.size()))
    
        
        print('----------------weight data for this step------------------------------------------------------')
        #print(hidden)
        print(self.linear_h.weight.data)
        
        print('----------------bias data for this step------------------------------------------------------')
        #print(hidden)
        print(self.linear_h.bias.data)        
        
        #print(self.linear_h.weight.data)
        #*activate.transpose(0,1)
        hidden_next = self.linear_h(hidden) + self.linear_i(x_input) #should be batch_size x hidden_dim
        
        print('----------------hidden_next data for this step------------------------------------------------------')
        #print(hidden)
        print(hidden_next) 
        
        hidden_next.data = activate*hidden_next.data + activate_re*hidden.data
        
        print('----------------hidden_next data for this step------------------------------------------------------')
        #print(hidden)
        print(hidden_next) 
        
        hidden_next = self.activation_fun(hidden_next)
        return hidden_next

    def init_hidden(self):
        h0 = Variable(torch.zeros(self.batch_size,self.num_units))
        return h0
        
    def initial_weights(self, mean, std):
        lin_layers = [self.linear_h, self.linear_o, self.linear_i]
        for layer in lin_layers:
            layer.weight.data.normal_(mean, std**2)
            layer.bias.data.fill_(0) 

In [ ]:
### Training
model = Clock_NN([1,2,4], batch_size, group_size = 5, activation_fun = F.relu, mean = 0, std = 0.1, input_dim = 5)

loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size)
for epoch in range(num_epochs):
    for step, (data, label) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        model.zero_grad()
        hidden= model.init_hidden()
        hidden, output = model(data, hidden)
        
        #now get a list of hidden and a list of outputs
        label = label.transpose(0,1).contiguous().view(-1) 
        #should be flatten, batch_size x hidden. transpose due to below order, was batch, seq => follow up 2 down. get size batch*seq
        output = torch.stack(output, dim=1).view(-1, 2) 
        #since batch is the first dimension, so dim is 1. now the order is  seq_len,batch, 2, so the first dimension is the first time step over all batch
        #print(output.size())
        #print(label.size())
        lossy = loss(output, label)
        lossy.backward()
        model.linear_h.weight.grad.data = model.linear_h.weight.grad.data*model.connect.transpose(0,1)
        #print(lossy.data[0])
        optimizer.step()
